In [1]:
# https://sites.google.com/site/nttrungmtwiki/home/it/machine-lear/neural-networks/develop-your-first-neural-network-in-python-with-keras-step-by-step
from __future__ import print_function
import librosa
import numpy as np
import os
import sys
import argparse

import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, Dense
from keras.optimizers import RMSprop, Adam, SGD
from keras import backend as K

from mirex2016_dataset import load_trainXY


Using TensorFlow backend.


In [2]:
#datadir = './dataset/mirex_beat_tracking_2016/train/x862x1025_y40x862/'
datadir = './x862x128_y862x1/'


In [3]:
trainXY_path = datadir
#tb_logdir = args.logdir
epochs = 500
learning_rates = [10, 1, 0.1, 0.01, 0.006, 0.003, 0.001, 0.0006, 0.0003, 0.0001, 0.00005, 0.00001]
loss_function = 'binary_crossentropy'

In [4]:
trainX, trainY, n_of_frames, n_of_freq_bins = load_trainXY(trainXY_path)
#trainX.resize(n_of_frames, 1, n_of_freq_bins)
#trainY.resize(n_of_frames, 1, 1)
trainX.resize(trainX.shape[0], 1, n_of_freq_bins)
trainY.resize(trainY.shape[0], 1, 1)
#trainX.resize(n_of_frames*2, 1, n_of_freq_bins)
#trainY.resize(n_of_frames*2, 1, 1)


Load trainX.npy and trainY.npy (21, 128)


In [5]:
trainXP = trainX.reshape(trainX.shape[0],n_of_freq_bins)
trainYP = trainY.reshape(trainY.shape[0],1)
print('trainXP.shape: ' + str(trainXP.shape))
print('trainYP.shape: ' + str(trainYP.shape))



trainXP.shape: (17240, 128)
trainYP.shape: (17240, 1)


In [6]:
import pandas as pd
trainXPD  = np.diff(trainXP,axis=0)
trainYPD  = trainYP[1:]
trainXPDH = np.maximum(np.zeros(trainXPD.shape),trainXPD)
print(trainXPD.shape)
xx = pd.DataFrame(trainXP[0:4][:])
dd = pd.DataFrame(trainXPD[0:3][:])
hh = pd.DataFrame(trainXPDH[0:5][:])
display(xx)
print('\n')
display(dd)
print('\n')
display(hh)

(17239, 128)


,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,...,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.0,-80.0,-80.0
1,-78.416825,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,...,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.0,-80.0,-80.0
2,-16.782757,-12.069625,-7.636879,-10.055699,-12.186193,-14.297897,-21.732388,-26.662192,-26.433747,-27.860949,...,-55.524676,-55.514602,-53.402921,-55.703926,-58.417543,-63.255391,-72.216577,-80.0,-80.0,-80.0
3,-3.696009,-6.775470,-12.168939,-16.854252,-18.960096,-24.425571,-27.194247,-27.391886,-28.151292,-35.199834,...,-37.081288,-42.817498,-44.738287,-43.906941,-43.328234,-54.360452,-54.851924,-80.0,-80.0,-80.0


,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,1.583175,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,61.634068,67.930375,72.363121,69.944301,67.813807,65.702103,58.267612,53.337808,53.566253,52.139051,...,24.475324,24.485398,26.597079,24.296074,21.582457,16.744609,7.783423,0.0,0.0,0.0
2,13.086748,5.294155,-4.532059,-6.798553,-6.773904,-10.127674,-5.461859,-0.729694,-1.717545,-7.338885,...,18.443388,12.697104,8.664634,11.796985,15.089309,8.894939,17.364653,0.0,0.0,0.0


,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,1.583175,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,61.634068,67.930375,72.363121,69.944301,67.813807,65.702103,58.267612,53.337808,53.566253,52.139051,...,24.475324,24.485398,26.597079,24.296074,21.582457,16.744609,7.783423,0.0,0.0,0.0
2,13.086748,5.294155,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,18.443388,12.697104,8.664634,11.796985,15.089309,8.894939,17.364653,0.0,0.0,0.0
3,0.000000,0.000000,0.739930,0.000000,0.000000,0.000000,0.000000,0.000000,1.397148,5.312010,...,0.000000,1.089796,0.000000,1.769560,0.000000,0.000000,0.000000,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.804787,...,0.000000,1.297082,4.546216,0.000000,0.000000,26.797198,0.000000,0.0,0.0,0.0


In [7]:
def f1_score(y_true, y_pred):
    #Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))
    #If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
         return 0
    # How many selected items are relevant?
    precision = c1 / c2
    recall = c1 / c3
    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

def recall(y_true, y_pred):
    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))
    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0
    recall = c1 / c3
    return recall

In [8]:
# recall rate could be 1.0 when
# 1 song/ 1 listenter
# lr = 0.0005
# epoch = 500
# bach  = 50
# first layer dence = 10 or 5 

# best bash_size = 150
lr = 0.000001
lr = 0.001
X = trainXPDH
Y = trainYPD
# create model
model = Sequential()
model.add(Dense(15, input_dim=n_of_freq_bins, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='relu'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',recall])
# Fit the model
model.fit(X, Y, nb_epoch=500, batch_size=150)
# evaluate the model
scores = model.evaluate(X, Y,verbose=0)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(15, kernel_initializer="uniform", activation="relu", input_dim=128)`
  from ipykernel import kernelapp as app
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="uniform", activation="relu")`
  app.launch_new_instance()
/usr/local/lib/python3.5/dist-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/500
17239/17239 [==============================] - 0s - loss: 1.4675 - acc: 0.5047 - recall: 0.2838     
Epoch 2/500
17239/17239 [==============================] - 0s - loss: 1.0289 - acc: 0.5189 - recall: 0.2914     
Epoch 3/500
17239/17239 [==============================] - 0s - loss: 0.9964 - acc: 0.5016 - recall: 0.1556     
Epoch 4/500
17239/17239 [==============================] - 0s - loss: 0.9692 - acc: 0.5235 - recall: 0.3198     
Epoch 5/500
17239/17239 [==============================] - 0s - loss: 0.9152 - acc: 0.5242 - recall: 0.2603     
Epoch 6/500
17239/17239 [==============================] - 0s - loss: 0.8424 - acc: 0.5232 - recall: 0.2158     
Epoch 7/500
17239/17239 [==============================] - 0s - loss: 0.8199 - acc: 0.5279 - recall: 0.2281     
Epoch 8/500
17239/17239 [==============================] - 0s - loss: 0.8227 - acc: 0.5201 - recall: 0.2018     
Epoch 9/500
17239/17239 [==============================] - 0s - loss: 0.7825 - acc: 0.5026 - rec

17239/17239 [==============================] - 0s - loss: 0.6276 - acc: 0.6378 - recall: 0.8644     
Epoch 73/500
17239/17239 [==============================] - 0s - loss: 0.6262 - acc: 0.6394 - recall: 0.8643     
Epoch 74/500
17239/17239 [==============================] - 0s - loss: 0.6248 - acc: 0.6412 - recall: 0.8647     
Epoch 75/500
17239/17239 [==============================] - 0s - loss: 0.6243 - acc: 0.6408 - recall: 0.8653     
Epoch 76/500
17239/17239 [==============================] - 0s - loss: 0.6245 - acc: 0.6408 - recall: 0.8645     
Epoch 77/500
17239/17239 [==============================] - 0s - loss: 0.6254 - acc: 0.6412 - recall: 0.8587     
Epoch 78/500
17239/17239 [==============================] - 0s - loss: 0.6213 - acc: 0.6429 - recall: 0.8611     
Epoch 79/500
17239/17239 [==============================] - 0s - loss: 0.6218 - acc: 0.6429 - recall: 0.8634     
Epoch 80/500
17239/17239 [==============================] - 0s - loss: 0.6211 - acc: 0.6414 - recall:

17239/17239 [==============================] - 0s - loss: 0.5827 - acc: 0.6683 - recall: 0.8359     
Epoch 144/500
17239/17239 [==============================] - 0s - loss: 0.5829 - acc: 0.6698 - recall: 0.8417     
Epoch 145/500
17239/17239 [==============================] - ETA: 0s - loss: 0.5799 - acc: 0.6724 - recall: 0.84 - 0s - loss: 0.5842 - acc: 0.6688 - recall: 0.8423     
Epoch 146/500
17239/17239 [==============================] - 0s - loss: 0.5925 - acc: 0.6657 - recall: 0.8460     
Epoch 147/500
17239/17239 [==============================] - 0s - loss: 0.5883 - acc: 0.6630 - recall: 0.8496     
Epoch 148/500
17239/17239 [==============================] - 0s - loss: 0.5919 - acc: 0.6640 - recall: 0.8490     
Epoch 149/500
17239/17239 [==============================] - 0s - loss: 0.5862 - acc: 0.6649 - recall: 0.8378     
Epoch 150/500
17239/17239 [==============================] - 0s - loss: 0.5927 - acc: 0.6619 - recall: 0.8366     
Epoch 151/500
17239/17239 [=============

17239/17239 [==============================] - 0s - loss: 0.5736 - acc: 0.6720 - recall: 0.8491     
Epoch 214/500
17239/17239 [==============================] - 0s - loss: 0.5753 - acc: 0.6699 - recall: 0.8530     
Epoch 215/500
17239/17239 [==============================] - 0s - loss: 0.5719 - acc: 0.6730 - recall: 0.8557     
Epoch 216/500
17239/17239 [==============================] - 0s - loss: 0.5713 - acc: 0.6727 - recall: 0.8459     
Epoch 217/500
17239/17239 [==============================] - 0s - loss: 0.5691 - acc: 0.6725 - recall: 0.8480     
Epoch 218/500
17239/17239 [==============================] - 0s - loss: 0.5688 - acc: 0.6715 - recall: 0.8455     
Epoch 219/500
17239/17239 [==============================] - 0s - loss: 0.5677 - acc: 0.6722 - recall: 0.8471     
Epoch 220/500
17239/17239 [==============================] - 0s - loss: 0.5662 - acc: 0.6734 - recall: 0.8499     
Epoch 221/500
17239/17239 [==============================] - 0s - loss: 0.5675 - acc: 0.6733 -

17239/17239 [==============================] - 0s - loss: 0.5839 - acc: 0.6705 - recall: 0.8712     
Epoch 285/500
17239/17239 [==============================] - 0s - loss: 0.5758 - acc: 0.6742 - recall: 0.8640     
Epoch 286/500
17239/17239 [==============================] - 0s - loss: 0.5702 - acc: 0.6742 - recall: 0.8681     
Epoch 287/500
17239/17239 [==============================] - 0s - loss: 0.5683 - acc: 0.6749 - recall: 0.8654     
Epoch 288/500
17239/17239 [==============================] - 0s - loss: 0.5665 - acc: 0.6767 - recall: 0.8661     
Epoch 289/500
17239/17239 [==============================] - 0s - loss: 0.5644 - acc: 0.6762 - recall: 0.8653     
Epoch 290/500
17239/17239 [==============================] - 0s - loss: 0.5644 - acc: 0.6753 - recall: 0.8618     
Epoch 291/500
17239/17239 [==============================] - 0s - loss: 0.5638 - acc: 0.6771 - recall: 0.8622     
Epoch 292/500
17239/17239 [==============================] - 0s - loss: 0.5633 - acc: 0.6774 -

17239/17239 [==============================] - 0s - loss: 0.5938 - acc: 0.6733 - recall: 0.8604     
Epoch 356/500
17239/17239 [==============================] - 0s - loss: 0.5914 - acc: 0.6733 - recall: 0.8649     
Epoch 357/500
17239/17239 [==============================] - 0s - loss: 0.5869 - acc: 0.6719 - recall: 0.8659     
Epoch 358/500
17239/17239 [==============================] - 0s - loss: 0.5835 - acc: 0.6735 - recall: 0.8672     
Epoch 359/500
17239/17239 [==============================] - 0s - loss: 0.5806 - acc: 0.6739 - recall: 0.8666     
Epoch 360/500
17239/17239 [==============================] - 0s - loss: 0.5812 - acc: 0.6742 - recall: 0.8692     
Epoch 361/500
17239/17239 [==============================] - 0s - loss: 0.5792 - acc: 0.6738 - recall: 0.8706     
Epoch 362/500
17239/17239 [==============================] - 0s - loss: 0.5770 - acc: 0.6746 - recall: 0.8678     
Epoch 363/500
17239/17239 [==============================] - 0s - loss: 0.5761 - acc: 0.6757 -

17239/17239 [==============================] - 0s - loss: 0.5695 - acc: 0.6683 - recall: 0.8689     
Epoch 427/500
17239/17239 [==============================] - 0s - loss: 0.5731 - acc: 0.6679 - recall: 0.8657     
Epoch 428/500
17239/17239 [==============================] - 0s - loss: 0.5724 - acc: 0.6675 - recall: 0.8651     
Epoch 429/500
17239/17239 [==============================] - 0s - loss: 0.5691 - acc: 0.6668 - recall: 0.8705     
Epoch 430/500
17239/17239 [==============================] - 0s - loss: 0.5690 - acc: 0.6680 - recall: 0.8662     
Epoch 431/500
17239/17239 [==============================] - 0s - loss: 0.5662 - acc: 0.6694 - recall: 0.8699     
Epoch 432/500
17239/17239 [==============================] - 0s - loss: 0.5690 - acc: 0.6692 - recall: 0.8660     
Epoch 433/500
17239/17239 [==============================] - 0s - loss: 0.5741 - acc: 0.6672 - recall: 0.8671     
Epoch 434/500
17239/17239 [==============================] - 0s - loss: 0.5749 - acc: 0.6675 -

17239/17239 [==============================] - 0s - loss: 0.5985 - acc: 0.6672 - recall: 0.8724     
Epoch 498/500
17239/17239 [==============================] - 0s - loss: 0.5840 - acc: 0.6652 - recall: 0.8732     
Epoch 499/500
17239/17239 [==============================] - 0s - loss: 0.5843 - acc: 0.6693 - recall: 0.8742     
Epoch 500/500
17239/17239 [==============================] - 0s - loss: 0.5793 - acc: 0.6676 - recall: 0.8785     

acc: 66.99%
